## Special Cases

this tweet is kind of a self reply: https://twitter.com/goblinodds/status/1539729495883583488?s=20&t=nlRVBzUc8rFeQx8LLLPaEg, but it also includes another user, so the code currently counts it as a reply only, which I am okay with for now... 



In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("self-reply.csv")

In [5]:
df["tweet_link"] = df.apply(lambda row: f"https://twitter.com/{row['author.username']}/status/{row.id}", axis=1)
print(df["tweet_type"])
df.iloc[1].tweet_link


0    self-reply
1         reply
2    self-reply
3         reply
Name: tweet_type, dtype: object


'https://twitter.com/goblinodds/status/1539729495883583488'

In [7]:
import json 
df["entities.mentions"].iloc[1]

mentions = json.loads(df["entities.mentions"].iloc[1])
mentions

[{'start': 0,
  'end': 13,
  'username': 'fun_vampire0',
  'id': '1443175286581911558',
  'profile_image_url': 'https://pbs.twimg.com/profile_images/1506300326617432072/WpiGhMxo_normal.jpg',
  'name': 'part of your in group',
  'protected': False,
  'url': '',
  'location': 'Uqbar ',
  'created_at': '2021-09-29T11:26:40.000Z',
  'verified': False,
  'public_metrics': {'followers_count': 194,
   'following_count': 1156,
   'tweet_count': 9002,
   'listed_count': 3},
  'description': 'I used to be someone, now I am someone else. I retweet stupid things, and make annoying observations.'}]

In [8]:
df.columns

Index(['Unnamed: 0', 'id', 'conversation_id',
       'referenced_tweets.replied_to.id', 'referenced_tweets.retweeted.id',
       'referenced_tweets.quoted.id', 'author_id', 'in_reply_to_user_id',
       'retweeted_user_id', 'quoted_user_id', 'created_at', 'text', 'lang',
       'source', 'public_metrics.like_count', 'public_metrics.quote_count',
       'public_metrics.reply_count', 'public_metrics.retweet_count',
       'reply_settings', 'possibly_sensitive', 'withheld.scope',
       'withheld.copyright', 'withheld.country_codes', 'entities.annotations',
       'entities.cashtags', 'entities.hashtags', 'entities.mentions',
       'entities.urls', 'context_annotations', 'attachments.media',
       'attachments.media_keys', 'attachments.poll.duration_minutes',
       'attachments.poll.end_datetime', 'attachments.poll.id',
       'attachments.poll.options', 'attachments.poll.voting_status',
       'attachments.poll_ids', 'author.id', 'author.created_at',
       'author.username', 'author.

In [ ]:
def extract_tweet_type(row):
    """
    Current Categories are 
    reply - reply that doesn't include a quoted tweet (I could expand to include mentions, but I don't feel like it right now)                 
    rt                     
    qt,reply - reply tweet that includes a quoted tweet
    standalone - classic tweet, just text          
    self-reply - reply to yourself          
    qt                     
    standalone,mention - classic tweet, but you also mention another user  
    qt,self-reply - quote tweet in reply to your own account      
    qt,mention - quote tweet where you mention another account, not a reply 
    """
    standalone = "standalone,"
    reply = "reply,"
    rt = "rt,"
    qt = "qt,"
    mention = "mention,"
    
    type_str = ""
    
    ## Standalone means that there is no in_reply_to_user_id or a referenced_tweet
    if (
        pd.isna(row["in_reply_to_user_id"]) and 
        pd.isna(row['referenced_tweets.replied_to.id']) and 
        pd.isna(row['referenced_tweets.retweeted.id']) and 
        pd.isna(row['referenced_tweets.quoted.id'])
    ):
        type_str += standalone
        
        if not pd.isna(row["entities.mentions"]):
            type_str += mention
    
    ## I think this is a special case where user starts there own standlone tweet with a mention, so twitter reads it weird
    # Here is an example: https://twitter.com/deepfates/status/1536434435075280898, it starts with a mention, with twitter reads into the in_reply_to_user_id field
    elif pd.isna(row['referenced_tweets.replied_to.id']) and not pd.isna(row["in_reply_to_user_id"]):
        type_str += standalone
        type_str += mention 
    
    ## Retweet
    elif not pd.isna(row['referenced_tweets.retweeted.id']):
        type_str += rt 
    
    ## Quote Tweet 
    elif not pd.isna(row['referenced_tweets.quoted.id']):
        type_str += qt 
        
        ## Quote Tweet that is a reply 
        if not pd.isna(row['referenced_tweets.replied_to.id']):
            if pd.isna(row["entities.mentions"]):
                type_str += "self-reply,"
            else:
                type_str += reply
        elif not pd.isna(row["entities.mentions"]): ## This logic is shaky
            type_str += mention
    
    ## Replies
    elif not pd.isna(row['referenced_tweets.replied_to.id']):
        if pd.isna(row["entities.mentions"]):
            type_str += "self-reply,"
        else:
            type_str += reply
        
    return type_str[:-1]